## Using Komo for IK

In [1]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry
import time
import cv2 as cv

** INFO:/home/tim/git/robotics-course_tim/rai/rai/Core/util.cpp:initCmdLine:602(1) ** cmd line arguments: 'rai-pybind -python'
**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Core/util.cpp:initCmdLine:606(1) ** run path: '/home/tim/git/robotics-course_tim/course3-Simulation/04-opencv'
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Core/graph.cpp:initParameters:1365(1) ** parsed parameters:
{python}


** INFO:/home/tim/git/robotics-course_tim/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback


In [2]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
D = C.view()

q_0 = C.getJointState()

In [3]:
obj = C.addFrame("object")
obj.setPosition([.3,0,.73])
#obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.ssBox, [.2,.05,0.04,0.01])
obj.setColor([1,1,0])
obj.setContact(1)


In [4]:
mk_obj = C.addFrame("mk_obj","object")
mk_obj.setShape(ry.ST.marker,[0.2])


In [5]:
mk_gripper = C.addFrame("mk_gripper","R_gripper")
mk_gripper.setShape(ry.ST.marker,[0.1])

mk_real = C.addFrame("mk_real_gripper")
mk_real.setShape(ry.ST.marker,[.1])


In [6]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/ex02_4.g")
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")

cameraFrame = C.frame("camera")

In [7]:
#the focal length
f = 0.895
f = f * 360.
#the relative pose of the camera
# pcl.setRelativePose('d(-90 0 0 1) t(-.08 .205 .115) d(26 1 0 0) d(-1 0 1 0) d(6 0 0 1) ')
fxfypxpy = [f, f, 320., 180.]

In [8]:
#-- the following is the simulation loop
tau = .01

for t in range(50):
    time.sleep(0.005)
    q = S.get_q()
    [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
    points = S.depthData2pointCloud(depth, fxfypxpy)
    cameraFrame.setPointCloud(points, rgb)
    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    C.setFrameState(RealWorld.getFrameState("obj1"),["object"])
    #send velocity controls to the simulation
    S.step(np.zeros_like(q), tau, ry.ControlMode.velocity)

In [9]:


# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,30, 5., True)
komo.addObjective([1.], ry.FS.positionDiff, ["R_gripper", "object"], ry.OT.sos, [1e2])
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)
komo.addObjective([.7,1.], ry.FS.position, ["R_gripper"], ry.OT.eq, [1e2],[-.0,-.1,-.0], order=2)
komo.addObjective([], ry.FS.distance, ["R_gripper", "object"], ry.OT.ineq, [1e2],[-.0])
komo.addObjective([1.], 
                ry.FS.scalarProductXX, 
                ["object","R_gripper"], 
                ry.OT.eq, 
                [1e2],
                [0]);
komo.addObjective([1.], 
                ry.FS.scalarProductZZ, 
                ["object","R_gripper"], 
                ry.OT.eq, 
                [1e2],
                [1]);
print(komo.getT(),komo.getPathFrames().shape)

komo.optimize()
#komo.getReport()
#V = komo.view_play(False, 1)



30 (30, 101, 7)
** KOMO::run solver:sparse collisions:1 x-dim:420 T:30 k:2 phases:1 stepsPerPhase:30 tau:0.166667  #timeSlices:32 #totalDOFs:420 #frames:3232
** optimization time:0.262848 (kin:0.005734 coll:0.18391 feat:0.036736 newton: 0.017726) setJointStateCount:27
   sos:4.85679 ineq:0.017037 eq:0.0151144


In [11]:
background = None

for frame,tau in zip(komo.getPathFrames(),komo.getPathTau()):
    time.sleep(tau)
    
    [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
    points = S.depthData2pointCloud(depth, fxfypxpy)
    cameraFrame.setPointCloud(points, rgb)

    background = track_background(depth,background)
    
    C.setFrameState(frame)
    
    if len(rgb)>0: cv.imshow('OPENCV - rgb', cv.cvtColor(rgb, cv.COLOR_BGR2RGB))
    
    if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)

    q= C.getJointState()

    #send position to the simulation
    S.step(q, tau, ry.ControlMode.position)

In [1]:
cv.destroyAllWindows()

NameError: name 'cv' is not defined

In [ ]:
import os
os._exit(0)